In [8]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

In [12]:
import pickle
with open("modelupdate.pkl", "rb") as f:
    model = pickle.load(f)
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:              como.rate   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     9.590
Date:                Sun, 27 Apr 2025   Prob (F-statistic):           8.79e-05
Time:                        18:40:55   Log-Likelihood:                 398.91
No. Observations:                 358   AIC:                            -791.8
Df Residuals:                     355   BIC:                            -780.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0172      0.063     -0.271

In [13]:
import geopandas as gpd

all_merged = gpd.read_file("allmerged.gpkg")
print(all_merged)

         geocode bufferzone  como.rate  veg_coverage veg_cat  heat_coverage  \
0    42101039100     Inside   0.000000      0.105444    High       0.334663   
1    42101980902     Inside   0.000000      0.073512  Medium       0.325128   
2    42101980904    Outside   0.000000      0.044030  Medium       0.350612   
3    42101980903     Inside   0.000000      0.053183  Medium       0.343371   
4    42101980906    Outside   0.000000      0.046935  Medium       0.339384   
..           ...        ...        ...           ...     ...            ...   
403  42101013602    Outside   0.143333      0.048246  Medium       0.324696   
404  42101034502    Outside   0.279412      0.059248  Medium       0.364994   
405  42101000902     Inside   0.225490      0.072492  Medium       0.303564   
406  42101001201     Inside   0.132701      0.066854  Medium       0.312582   
407  42101002702     Inside   0.198238      0.055700  Medium       0.323058   

     pred_como.rate                                

In [31]:
# Define a threshold for 'high risk' based on predicted values
threshold= 0.375 #top quartile based on SD
high_risk_threshold = all_merged['pred_como.rate'].quantile(0.95)  
all_merged['high_risk'] = all_merged['pred_como.rate'] > high_risk_threshold
# Loop through GeoDataFrame to identify areas based on heat coverage and comorbidity rate
for idx, row in all_merged.iterrows():
    if row['heat_coverage'] > threshold and row['pred_como.rate'] > high_risk_threshold:
        print(f"Region {row['geocode']} is at high risk.")


Region 42101011700 is at high risk.
Region 42101029200 is at high risk.
Region 42101028400 is at high risk.
Region 42101012201 is at high risk.
Region 42101009802 is at high risk.
Region 42101037500 is at high risk.
Region 42101011000 is at high risk.
Region 42101011100 is at high risk.
Region 42101011400 is at high risk.
Region 42101012000 is at high risk.
Region 42101012100 is at high risk.
Region 42101980500 is at high risk.
Region 42101020200 is at high risk.
Region 42101020300 is at high risk.
Region 42101020500 is at high risk.
Region 42101980002 is at high risk.
Region 42101989300 is at high risk.
Region 42101017000 is at high risk.
Region 42101980800 is at high risk.
Region 42101010800 is at high risk.
Region 42101012204 is at high risk.


In [32]:
all_merged['high_risk'] = (
    (all_merged['heat_coverage'] > threshold) &
    (all_merged['pred_como.rate'] > high_risk_threshold)
)


In [34]:
visual_data = all_merged.copy()

# Convert all categorical columns to strings
for col in visual_data.select_dtypes(include='category').columns:
    visual_data[col] = visual_data[col].astype(str)

# Now save safely
visual_data.to_file("visual_data.gpkg", driver="GPKG")